In [35]:
import numpy as np
import optuna
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.preprocessing import MaxAbsScaler

from darts.dataprocessing.transformers import Scaler
from darts.datasets import AirPassengersDataset
from darts.metrics import smape
from darts.models import TCNModel
from darts.utils.likelihood_models import GaussianLikelihood

import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt

from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel
from darts.datasets import WeatherDataset
from darts.models import LinearRegressionModel
import darts.metrics as metrics
from darts.datasets import AirPassengersDataset
import datetime

import numpy as np
import optuna
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.preprocessing import MaxAbsScaler

from darts.dataprocessing.transformers import Scaler
from darts.datasets import AirPassengersDataset
from darts.models import TCNModel
from darts.utils.likelihood_models import GaussianLikelihood
from optuna.terminator import report_cross_validation_scores

from darts.models import TFTModel
from darts.utils.likelihood_models import QuantileRegression
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from darts import TimeSeries

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

from scikeras.wrappers import KerasClassifier
from model_base import *

In [36]:
bld_pd=pd.read_csv(r'/root/autodl-tmp/data/load_prediction_base/BLD_Sum.csv')
bld_pd.sort_values(by='DateTime')

bld_target_pd=bld_pd[['RealPower','DateTime']]
bld_past_co_pd=bld_pd[['DateTime','temp','feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity']]
bld_future_co_pd=bld_pd[['DateTime','hour_cos', 'hour_sin','dayofweek_cos', 'dayofweek_sin',
       'month_cos', 'month_sin', 'dayofmonth_cos', 'dayofmonth_sin']]


bld_target=TimeSeries.from_dataframe(bld_target_pd,time_col="DateTime",freq="15min",fill_missing_dates=True)
bld_past_co=TimeSeries.from_dataframe(bld_past_co_pd,time_col="DateTime",freq="15min",fill_missing_dates=True)
bld_future_co=TimeSeries.from_dataframe(bld_future_co_pd,time_col="DateTime",freq="15min",fill_missing_dates=True)

transformer = Scaler()

# data split
train_start=pd.Timestamp(2017,1,1,0,0)
train_end=pd.Timestamp(2018,8,31,23,45)

val_start=pd.Timestamp(2018,9,1,0,0)
val_end=pd.Timestamp(2018,12,31,23,45)

pred_start=pd.Timestamp(2019,1,1,0,0)
pred_end=pd.Timestamp(2019,12,31,23,45)

In [37]:
bld_target

<TimeSeries (DataArray) (DateTime: 110879, component: 1, sample: 1)>
array([[[51.7251]],

       [[53.0934]],

       [[52.5827]],

       ...,

       [[69.5311]],

       [[66.1281]],

       [[64.0733]]])
Coordinates:
  * DateTime   (DateTime) datetime64[ns] 2017-01-01T00:15:00 ... 2020-02-29T2...
  * component  (component) object 'RealPower'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [38]:


'''
# load data
series = AirPassengersDataset().load().astype(np.float32)

# split in train / validation (note: in practice we would also need a test set)
VAL_LEN = 36
train, val = series[:-VAL_LEN], series[-VAL_LEN:]

# scale
scaler = Scaler(MaxAbsScaler())
train = scaler.fit_transform(train)
val = scaler.transform(val)
'''

# define objective function
def objective(trial):
    
    # detect if a GPU is available
    if torch.cuda.is_available():
        num_workers = 4
    else:
        num_workers = 0


    # reproducibility
    torch.manual_seed(42)
    
    my_stopper = EarlyStopping(
        monitor="train_loss",
        patience=5,
        min_delta=1e-1,
        mode='min',
    )

    # build the TCN model
    model = RNNModel(
        optimizer_kwargs={
            'lr':trial.suggest_float("lr", 1e-5, 1e-1),
        },
        pl_trainer_kwargs={
            "callbacks": [my_stopper],
            "gradient_clip_val":0.1,
            "accelerator": "gpu",
            "devices": [0]
            },
        model='LSTM',
        hidden_dim=trial.suggest_int("hidden_dim", 8, 256),
        n_rnn_layers=trial.suggest_int("n_rnn_layers", 1, 8),
        batch_size=trial.suggest_int("batch_size", 8, 96*2),
        dropout=trial.suggest_float("dropout", 0.01, 0.5),
        input_chunk_length=96*7,
        output_chunk_length=96,
        #'loss_fn':torch.nn.GaussianNLLLoss(reduction='mean'),
        #'likelihood':GaussianLikelihood(),
        # QuantileRegression(
        #    quantiles=quantiles
        #),
        n_epochs=trial.suggest_int("n_epochs", 1, 20),

        random_state=42,
        
    )


    # when validating during training, we can use a slightly longer validation
    # set which also contains the first input_chunk_length time steps
    #model_val_set = scaler.transform(series[-(VAL_LEN + in_len) :])

    model.fit(
        series=bld_target[train_start:train_end],
        future_covariates=bld_future_co[train_start:train_end],
        val_series=bld_target[val_start:val_end],
        val_future_covariates=bld_future_co[val_start:val_end],
        
        num_loader_workers=4
    )

    # reload best model over course of training
    #model = TCNModel.load_from_checkpoint("prob_rnn_model")

    # Evaluate how good it is on the validation set, using sMAPE
    preds = model.predict(    
                        n=len(bld_target[val_start:val_end]),
                        series=bld_target[train_start:train_end],
                        future_covariates=bld_future_co,
                        n_jobs=-1,
                        num_samples=1)
    '''    
    scores=cross_val_score(model, bld_target[train_start:train_end],
                            cv=KFold(n_splits=4,shuffle=True),
                            scoring='neg_mean_absolute_error')# bld_future_co[train_start:train_end], 
                            '''
    
    smapes = smape(bld_target, preds, n_jobs=-1, verbose=True,intersect=True)
    smape_val = np.mean(smapes)
    #return scores.mean()
    return smape_val if smape_val != np.nan else float("inf")


# for convenience, print some optimization trials information
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")


# optimize hyperparameters by minimizing the sMAPE on the validation set


In [39]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1, callbacks=[print_callback])

[I 2023-10-11 16:07:23,284] A new study created in memory with name: no-name-1ba80bb3-4417-4f6c-a167-e696d3cb2b5c
ignoring user defined `output_chunk_length`. RNNModel uses a fixed `output_chunk_length=1`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 93.8 K
4 | V             | Linear           | 68    
---------------------------------------------------
93.9 K    Trainable params
0         Non-trainable params
93.9 K    Total params
0.375     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[I 2023-10-11 16:10:13,818] Trial 0 finished with value: 20.160563249251233 and parameters: {'lr': 0.012417416273672465, 'hidden_dim': 67, 'n_rnn_layers': 3, 'batch_size': 125, 'dropout': 0.4301434163799506, 'n_epochs': 19}. Best is trial 0 with value: 20.160563249251233.


Current value: 20.160563249251233, Current params: {'lr': 0.012417416273672465, 'hidden_dim': 67, 'n_rnn_layers': 3, 'batch_size': 125, 'dropout': 0.4301434163799506, 'n_epochs': 19}
Best value: 20.160563249251233, Best params: {'lr': 0.012417416273672465, 'hidden_dim': 67, 'n_rnn_layers': 3, 'batch_size': 125, 'dropout': 0.4301434163799506, 'n_epochs': 19}


In [40]:
class Model_base():
    # set basic settings:
    # 1. model identification
    # 2. params of model, and whether call optuna
    # 3. prediction settings
    # 2. saving settings
    def __init__(self,id, model_type,
                 call_optuna, optuna_settings, optuna_params_dic,
                 call_one_trial, one_trial_settings, one_trial_params_dic,
                 load_from_trained_model,load_settings,
                 load_from_checkpoint,load_checkpoint_settings,
                 predict_settings,metric_settings, save_settings):
        # for optuna params_dic should be like:
        #   dic={
        #       'param_key':[tpye, lower_bound, upper_bound, bool_log], #params to search
        #       'param_key':[value] #params not to search
        #   }
        # check if necessary key pairs are passed
        
        for i in ['folder','save_model','save_prediction','save_metrics']:
            assert i in save_settings.keys()
            
        self.model_type=model_type
        self.model=None # tmp model for optuna / model load from file
        
        self.save_settings=save_settings
        self.save_path=os.path.join(save_settings['folder'],id)
        self.id=id
        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)
        
        self.call_optuna=call_optuna
        if call_optuna==True:
            assert optuna_settings!=None
            assert optuna_params_dic!=None
        self.optuna_params_dic=optuna_params_dic
        self.optuna_settings=optuna_settings
        self.optuna_model_name=None
        
        self.call_one_trial=call_one_trial
        if call_one_trial==True:
            assert one_trial_settings!=None
            assert one_trial_params_dic!=None
        self.one_trial_params_dic=one_trial_params_dic
        self.one_trial_settings=one_trial_settings
        self.one_trial_model_name=None
        
        self.load_from_trained_model=load_from_trained_model
        if load_from_trained_model==True:
            assert load_settings!=None
            self.load_settings=load_settings
            self.load_from_model()
            
        self.load_from_checkpoint=load_from_checkpoint
        if load_from_checkpoint==True:
            assert load_checkpoint_settings!=None
            self.load_checkpoint_settings=load_checkpoint_settings
            self.load_from_checkpoints()
        
        
        self.predict_settings=predict_settings
        self.metric_settings=metric_settings
        

        
        self.optuna_study=None
        self.best_params=None
        self.best_model=None
        self.best_model_name=None
        
        self.one_trial_model=None # model from run_one_trial
        
        ...
    def load_from_model(self):
        self.model=self.model_type.load(**self.load_settings)
        ...
        
    def load_from_checkpoint(self):
        self.model=self.model_type.load_from_checkpoint(**self.load_checkpoint_settings)
    # run one trial on designated params
    def run_one_trial(self):
        now=datetime.datetime.now()
        self.one_trial_model_name='-'.join([str(now.month),str(now.day),str(now.hour),str(now.minute),'one_trial'])
        
        self.one_trial_model=self.model_type(work_dir=self.save_path,
                                             model_name=self.one_trial_model_name, log_tensorboard=True,
                                             save_checkpoints=True,
                                             **self.one_trial_params_dic)
        #self.one_trial_model, optimizer = amp.initialize(self.one_trial_model, optimizer, opt_level='O1')

        self.one_trial_model.fit(**self.one_trial_settings)
        
    # run hyperparams searching
    def run_optuna(self):
        p=self.optuna_params_dic
        
        X_train=self.optuna_settings['X_train']
        y_train=self.optuna_settings['y_train']
        metric_cv=self.optuna_settings['metric_cv']
        n_trials=self.optuna_settings['n_trials']
        stop_threshold=self.optuna_settings['stop_threshold']
        direction=self.optuna_settings['direction']
        
        def objective(trial,p):
            params={}
            for i in p:
                
                if len(p[i])==1:
                    params.update({i,p[i][0]})
                elif len(p[i])==4:
                    assert isinstance(p[i][1],p[i][0])
                    assert isinstance(p[i][2],p[i][0])
                    if p[i][0]==int:
                        params.update({i,trial.suggest_int(i,p[i][1],p[i][2],p[i][3])})
                    elif p[i][0]==float:
                        params.update({i,trial.suggest_float(i,p[i][1],p[i][2],p[i][3])})
                    elif p[i][0]==bool:
                        params.update({i,trial.suggest_float(i,p[i][1],[p[i][2],p[i][3]])})
                    else:
                        Warning("Invalid param type!")
            self.model=self.model_type(**params)
            self.model.fit(**self.one_trial_settings)
            
            scores=cross_val_score(self.model, X_train, y_train,
                                   cv=KFold(n_split=5,shuffle=True),
                                   scoring=metric_cv)
            
            report_cross_validation_scores(trial, scores)
            return scores.mean()
        
        def callback(study, trial):
            for ii, t in enumerate(study.trials):
                if t.value >= stop_threshold:
                    study.stop()
                    
        study = optuna.create_study(directions=direction)
        
        study.optimize(objective, n_trials=n_trials, gc_after_trial=True,
                   callbacks=[callback])
    
        print('Number of finished trials: {}'.format(len(study.trials)))
        print('Best trial:')
        trial = study.best_trial

        print('  Value: {}'.format(trial.value))
        print('  Params: ')

        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))
            
        self.optuna_study=study
        return study
    
    def refit_best_trial(self):
        assert self.optuna_study!=None
        assert self.one_trial_params_dic!=None
        
        now=datetime.datetime.now()
        self.best_model_name='-'.join([str(now.month),str(now.day),str(now.hour),str(now.minute),'best_trial'])
        params=self.one_trial_params_dic.copy()
        best_trail=self.optuna_study.best_trial
        best_params=best_trail.params
        for i in best_params:
            if i=='lr':
                params['optimizer_kwargs'].update({i:best_params[i]})
            else:
                params.update({i:best_params[i]})
        #best_params['tree_method']='gpu_hist'
        self.best_params=params
        model=self.model_type(**self.best_params)
        model.fit(**self.one_trial_settings)
        self.best_model=model
        return model
    
    def save_one_trial_model(self):
        assert self.one_trial_model !=None
        model_path=os.path.join(self.save_path,self.one_trial_model_name,'trained_model')
        if not os.path.exists(model_path):
            os.makedirs(model_path)
        model_name=self.one_trial_model_name
        model_name=os.path.join(model_path,model_name+'.pt')
        self.one_trial_model.save(model_name)
        
    def save_best_model(self):
        assert self.best_model !=None
        model_path=os.path.join(self.save_path,self.best_model_name,'trained_model')
        if not os.path.exists(model_path):
            os.makedirs(model_path)
        model_name=self.best_model_name
        model_name=os.path.join(model_path,model_name+'.pt')
        self.best_model.save(model_name)
        
    def predict_on_one_trial_model(self):
        self.prediction=self.one_trial_model.predict(**self.predict_settings)
        self.prediction.to_csv(os.path.join(self.save_path,self.one_trial_model_name+'prediction.csv'))
        metrics_df=self.cal_metrics()
        metrics_df.to_csv(os.path.join(self.save_path,self.one_trial_model_name+'prediction_metrics.csv'))
        
    def predict_on_best_model(self):
        self.prediction=self.best_model.predict(**self.predict_settings)
        self.prediction.to_csv(os.path.join(self.save_path,self.best_model_name+'prediction.csv'))
        
    def predict_on_loaded_model(self):
        self.prediction=self.model.predict(**self.predict_settings)
        self.prediction.to_csv(os.path.join(self.save_path,'loaded_model_prediction.csv'))

    def cal_metrics(self):
        assert self.prediction!=None
        metrics_method_dic={
            'CV':metrics.coefficient_of_variation,
            'MAE':metrics.mae,
            'MAPE':metrics.mape,
            'OPE':metrics.ope,
            'RMSE':metrics.rmse,
            'MSE':metrics.mse,
            'MARRE':metrics.marre,
            'MASE':metrics.mase,
            'R2':metrics.r2_score,
            'SMAPE':metrics.smape,
        }
        metrics_dic={
            'start_time':self.prediction.time_index[0],
            'end_time':self.prediction.time_index[-1],
            'n':len(self.prediction.time_index),
        }
        
        for metric in metrics_method_dic.keys():
            try:
                if metric=='MASE':
                    value=metrics_method_dic[metric](self.metric_settings['series_pred_gt'],self.prediction,intersect=True,
                                                              insample=self.metric_settings['series_train'], m=96*7)
                    print({metric: value})
                    metrics_dic.update({metric: value})
                else:
                    value=metrics_method_dic[metric](self.metric_settings['series_pred_gt'],self.prediction,intersect=True)
                    print({metric: value})
                    metrics_dic.update({metric: value})
            except:
                print("Fail to calculate metric: {} of model {}".format(metric,self.id))
                
        metrics_df=pd.DataFrame([metrics_dic]).T
        return metrics_df

In [41]:
model_type=RNNModel
call_optuna=False
optuna_settings=None
optuna_params_dic=None
call_one_trial=False
quantiles = [
    0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.4,0.5,
    0.6,0.7,0.75,0.8,0.85,0.9,0.95,0.99,
]

one_trial_settings={
    'series':bld_target[train_start:train_end],
    'future_covariates':bld_future_co[train_start:train_end],
    'val_series':bld_target[val_start:val_end],
    'val_future_covariates':bld_future_co[val_start:val_end],
    'num_loader_workers':4
}

my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=5,
    min_delta=1e-1,
    mode='min',
)

one_trial_params_dic={
    'optimizer_kwargs':{
        'lr':1e-4,
    },
    'pl_trainer_kwargs':{
        "callbacks": [my_stopper],
        "gradient_clip_val":0.1},
    'model':'LSTM',
    'hidden_dim':128,
    'n_rnn_layers':4,
    'batch_size':64,
    'dropout':0.2,
    'input_chunk_length':96*7,
    'output_chunk_length':96,
    #'loss_fn':torch.nn.GaussianNLLLoss(reduction='mean'),
    #'likelihood':GaussianLikelihood(),
    # QuantileRegression(
    #    quantiles=quantiles
    #),
    'n_epochs':10,
    'pl_trainer_kwargs':{
      "accelerator": "gpu",
      "devices": [0]
    },
    'random_state':42,
}
predict_settings={
    'n':96*365,
    'series':bld_target[:pred_start],
    #'past_covariates':bld_past_co,
    'future_covariates':bld_future_co,
    'n_jobs':-1,
    #'num_samples':1
}
metric_settings={
    'series_pred_gt':bld_target[pred_start:pred_end],
    'series_train':bld_target[:pred_start],
}
save_settings={
    'folder':r'/root/autodl-tmp/load_forecast/DeepAR(Prob_RNN)',
    'save_model':True,'save_prediction':True,'save_metrics':True
}
test=Model_base(
    'test_ep_4',
    model_type,
    call_optuna,
    optuna_settings,
    optuna_params_dic,
    call_one_trial,
    one_trial_settings,
    one_trial_params_dic,
    False,
    None,
    False,
    None,
    predict_settings,
    metric_settings,
    save_settings
)
test.optuna_study=study


In [42]:
test.refit_best_trial()
test.predict_on_best_model()
test.save_best_model()

ignoring user defined `output_chunk_length`. RNNModel uses a fixed `output_chunk_length=1`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 93.8 K
4 | V             | Linear           | 68    
---------------------------------------------------
93.9 K    Trainable params
0         Non-trainable params
93.9 K    Total params
0.375     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=19` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [ ]:
prediction=test.best_model.historical_forecasts(
    series=bld_target[pd.Timestamp(2018,12,24,0,0):pred_end],
    future_covariates=bld_future_co,
    num_samples=1,
    start=pd.Timestamp(2018,12,31,0,0),
    forecast_horizon=96*7,
    stride=96*7,
    retrain=False,
    verbose=True,
    last_points_only=False
)

In [49]:
prediction[1]

<TimeSeries (DataArray) (DateTime: 672, component: 1, sample: 1)>
array([[[73.64331395]],

       [[75.53570829]],

       [[77.04352057]],

       [[77.32667919]],

       [[77.56805357]],

       [[77.65169534]],

       [[77.78346695]],

       [[77.89171083]],

       [[77.93682162]],

       [[77.92292508]],

...

       [[85.26915166]],

       [[85.27768174]],

       [[85.17930107]],

       [[84.72172244]],

       [[84.48596484]],

       [[84.51365096]],

       [[83.87320313]],

       [[82.15751921]],

       [[82.16514657]],

       [[84.85920909]]])
Coordinates:
  * DateTime   (DateTime) datetime64[ns] 2019-01-07 ... 2019-01-13T23:45:00
  * component  (component) object 'RealPower'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [51]:
temp=prediction[0]

for i in range(len(prediction)-1):
    temp=temp.concatenate(prediction[i+1])

In [53]:
test.prediction=temp

In [54]:
test.cal_metrics()

ValueError: The pred_series must be the forecast of the insample series


{'CV': 24.92620854067723}
{'MAE': 15.752964499993059}
{'MAPE': 24.95025960235859}
{'OPE': 14.799577300839948}
{'RMSE': 18.12751541918411}
{'MSE': 328.6068152727577}
{'MARRE': 21.942170626415436}
Fail to calculate metric: MASE of model test_ep_4
{'R2': -0.2935144814285069}
{'SMAPE': 21.346751950540497}


,0
start_time,2018-12-31 00:00:00
end_time,2019-12-29 23:45:00
n,34944
CV,24.926209
MAE,15.752964
MAPE,24.95026
OPE,14.799577
RMSE,18.127515
MSE,328.606815
MARRE,21.942171
